In [3]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn import ensemble
from sklearn import linear_model
import seaborn as sns
import matplotlib.pyplot as plt
from functions import get_baskets, get_coupons, get_prediction_index, get_3_files
from functions import split_4_way
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
takes = lambda n_old, n_new, time_old: print(f'Will take {int((time_old/n_old*n_new)/60)}m {int((time_old/n_old*n_new)%60)}s') 

# Loading datasets

In [3]:
baskets, coupons, prediction_index = get_3_files()
customers = (1000,2000)
n_customers = customers[1] - customers[0]
baskets = baskets[(baskets['customer'] > customers[0]) & (baskets['customer'] < customers[1])]

# Creating: Universal base

In [3]:
def create_base(weeks=range(90), customers=range(2000)):
    products = range(250)
    n_weeks, n_customers, n_products = len(weeks), len(customers), len(products)

    base = pd.DataFrame({
    'week': np.array([[x] * n_products * n_customers for x in weeks]).flatten(),
    'customer': np.array([[x] * n_products for x in customers] * n_weeks).flatten(),
    'product': list(range(n_products)) * n_customers * n_weeks
    })
    
    return base

# Adding: Basket information

In [4]:
def add_basket_info(base, baskets):
    base = pd.merge(base, baskets, on=['week', 'customer','product'], how='left')
    base['price'] = base['price'].fillna(0).astype(int)
    base['isBought'] = (base['price'] > 0)
    base['basket'] = base['week'].astype(str) + '_' + base['customer'].astype(str)
    return base

# Adding: Coupon information

In [5]:
def add_coupon_info(base, baskets, coupons):
    base = pd.merge(base, coupons, on=['week', 'customer','product'], how='left')
    base['discount'] = base['discount'].fillna(0).astype(int)
    base = base.rename(columns={"discount": "dGiven"})
    base['isGiven'] = (base['dGiven'] > 0)

    normal_prices = baskets.groupby('product')['price'].max().values
    base['highestPrice'] = base['product'].apply(lambda x: normal_prices[x])
    base['isUsed'] = ((base['price'] != base['highestPrice']) & (base['price']!=0))
    base.drop('highestPrice', axis=1, inplace=True)
    return base

# Finding: Categories

In [7]:

# base = create_base(customers=range(1000,2000),weeks=range(89))
# base = add_basket_info(base,baskets)
# 
# n_baskets = base['basket'].nunique()
# 
# matrix = np.matrix([np.array(base.iloc[x*250:x*250+250,:].loc[:,'isBought']) for x in range(n_baskets)]).astype(int)
# 
# occurency_matrix = matrix.T.dot(matrix)
# occurency_matrix[occurency_matrix > 1] = 1
# 
# sns.set(rc={'figure.figsize':(6,5)})
# sns.heatmap(occurency_matrix, annot=False)
# plt.savefig("full.png")


In [8]:
# sns.heatmap(occurency_matrix[:31,:31], annot=False)
# plt.savefig("part.png")

# Adding: Categories

In [9]:
def add_categories(base):
    base['category'] = (base['product'] / 10).astype(int)
    return base

# Adding: Probabilities (product & category)

In [11]:
def add_frequencies(base, baskets):
    
    baskets.loc[:,'category'] = (baskets['product'].astype(int) / 10).astype(int)
    n_weeks = baskets['week'].nunique()
    
    prod_probs_df = (
        baskets.groupby(['customer','product'])['week'].count() / n_weeks) \
        .reset_index() \
        .rename(columns={'week':'probability'})

    cat_probs_df = (
        baskets.groupby(['customer','category'])['week'].count() / n_weeks) \
        .reset_index() \
        .rename(columns={'week':'probability'})
        
    base = pd.merge(base, prod_probs_df, on=['customer','product'] ,how='left')
    base = pd.merge(base, cat_probs_df, on=['customer','category'] ,how='left')
    
    base.rename(columns={'probability_x':'p_prod', 'probability_y':'p_cat'},inplace=True)
    
    base[['p_prod','p_cat']] = base[['p_prod','p_cat']].fillna(0)
    
    return base

# Adding: Rolling frequencies

In [12]:
def get_rolling_frequencies(n_weeks=5, category=False):
    
    rolling_df = pd.DataFrame()
    
    for week_nr in range(n_weeks,89+1):

        start = week_nr - n_weeks - 1
        end = week_nr

        single_week = (
            base[(start < base['week']) & (base['week'] < end)] 
            .groupby(['customer',f"{'category' if category else 'product'}"]) 
            .agg({'week':'last','isBought':'sum'}) 
            .reset_index()
            )
        
        single_week['week'] = single_week['week'] + 1
        single_week['isBought'] = single_week['isBought'] / n_weeks
        rolling_df = pd.concat([rolling_df, single_week])
    
    return rolling_df

In [13]:
def add_rolling_frequencies(base):
    
    values = [5,10,30]
    prod_names = [f'roll_prod_{value}' for value in values]
    cat_names  = [f'roll_cat_{value}' for value in values]

    for i, name in enumerate(prod_names):
        rolled = get_rolling_frequencies(n_weeks=values[i]).rename(columns={'isBought':name})
        base = pd.merge(base, rolled, on=['week','customer','product'],how='left')

    for i, name in enumerate(cat_names):
        rolled = get_rolling_frequencies(n_weeks=values[i], category=True).rename(columns={'isBought':name})
        base2 = pd.merge(base, rolled, on=['week','customer','category'],how='left') 
        
    base.loc[:,prod_names[0]:] = base2.loc[:,prod_names[0]:].fillna(0)  
    
    return base

# Adding: Weeks since last buy (per product and per category)

In [14]:
def buy_weeks_to_ago(buy_weeks):
    weeks_past = 90
    val = []
    bought = False

    for i in range(90):
        if i in buy_weeks:
            weeks_past = 0
            val.append(weeks_past)
            bought = True
        elif(bought==True): 
            weeks_past += 1 
            val.append(weeks_past)
        else:
            val.append(90)
    return val

In [15]:
def add_weeks_ago(base):
    
    only_bought = base[base['isBought']==1]
    pairs = only_bought.groupby(['customer','product']).size().reset_index(inplace=False)

    all_pairs = pd.DataFrame()

    for i in range(len(pairs)):
        customer = pairs.iloc[i,0]
        product = pairs.iloc[i,1]
        
        df = pd.DataFrame({
            'week':range(90),
            'customer':customer, 
            'product':product, 
            })
        
        buy_weeks = list(only_bought[(only_bought['product']==product) & (only_bought['customer']==customer)]['week'])
        df['weeks_ago'] = buy_weeks_to_ago(buy_weeks)
        all_pairs = pd.concat([all_pairs,df])
        
    base = pd.merge(base, all_pairs, on=['week','customer','product'],how='left').fillna(90)
    
    return base

# Actually creating bases

In [17]:
unkown = 89
customers = (1500,2000)
n_customers = customers[1] - customers[0]
ranger = range(*customers)

In [18]:
baskets = get_baskets()
baskets = baskets[(baskets['customer'] > customers[0]) & (baskets['customer'] < customers[1])]
baskets = baskets[baskets['week']<unkown]

coupons = get_coupons()
coupons = coupons[(coupons['customer'] > customers[0]) & (coupons['customer'] < customers[1])]
coupons = coupons[coupons['week']<unkown]

## Training base

As the data is consecutive we will create a base dataframe for both training and testing in 1. Then we will split them into to seperate datasets. When eventually predicting week 90 we will do the same. We will however only use the baskets & coupons datasets that contain information about all the weeks before the week that needs predicting. Direct coupon and basket related values will likely be set to false or to 0.

In [19]:
base = create_base(customers=ranger,weeks=range(unkown+1))
base = add_basket_info(base, baskets)
base = add_coupon_info(base, baskets, coupons)
base = add_categories(base)
base = add_frequencies(base, baskets) #Will generate frequencies based on available basket info

In [119]:
takes(500,n_customers,240)
base = add_rolling_frequencies(base)

Will take 9m 25s


In [121]:
takes(100,n_customers,174)
#base = add_weeks_ago(base)

Will take 14m 30s


Some columns will be useless for predictions these are 'basket' and 'price'. Basket is a never before seen string. Price will be set to zero which is untrue we don't know how much they'll spend.

In [123]:
base = base.drop([
    'price',
    'basket'
    ],axis=1)

## Splitting bases

In [124]:
train = base[base['week']<unkown]
x_train = train.drop('isBought', axis=1)
y_train = train['isBought']

In [125]:
test = base[base['week']==unkown]
x_test = test.drop('isBought', axis=1)

In [126]:
forbidden_baskets = get_baskets()
forbidden_baskets = forbidden_baskets[forbidden_baskets['week']==unkown]

y_test = create_base(weeks=range(unkown, unkown+1),customers=ranger)
y_test = add_basket_info(y_test,baskets=forbidden_baskets)['isBought'].values

# Modelling

In [4]:
models = [
    ensemble.RandomForestClassifier(), #2.6%
    ensemble.GradientBoostingClassifier(), #1.9%
    ensemble.HistGradientBoostingClassifier() #2.4%
] 

In [5]:
#forbidden_columns = ['price','isUsed']
#x_train, y_train, x_test, y_test = split_4_way(base.drop(forbidden_columns, axis=1), target_col='isBought', unkwown_week=89)

for i, model in enumerate(models):
    
    print(f'Model {i}: {model}')
    
    model.fit(X = x_train, y = y_train)
    predictions = model.predict_proba(x_test)
    cross_entropy_loss = metrics.log_loss(y_test, predictions)
    
    print(f'Loss: {round(cross_entropy_loss,3)}\n')

Model 0: RandomForestClassifier()


NameError: name 'x_train' is not defined